In [13]:
from itertools import combinations

def unify(term1, term2, substitutions={}):
    """Unify two predicates using a substitution map."""
    if substitutions is None:
        return None
    elif term1 == term2:
        return substitutions
    elif isinstance(term1, str) and term1.islower():
        return unify_variable(term1, term2, substitutions)
    elif isinstance(term2, str) and term2.islower():
        return unify_variable(term2, term1, substitutions)
    elif isinstance(term1, tuple) and isinstance(term2, tuple) and len(term1) == len(term2):
        return unify(term1[1:], term2[1:], unify(term1[0], term2[0], substitutions))
    else:
        return None

def unify_variable(variable, value, substitutions):
    """Unify a variable with a value."""
    if variable in substitutions:
        return unify(substitutions[variable], value, substitutions)
    elif value in substitutions:
        return unify(variable, substitutions[value], substitutions)
    else:
        substitutions[variable] = value
        return substitutions

def resolve(clause_a, clause_b):
    """Resolve two clauses and return the resolvents."""
    resolvents = []
    for predicate_a in clause_a:
        for predicate_b in clause_b:
            substitutions = unify(predicate_a, negate(predicate_b))
            if substitutions is not None:
                new_clause = (apply_substitution(clause_a, substitutions) | apply_substitution(clause_b, substitutions)) - {predicate_a, predicate_b}
                resolvents.append(frozenset(new_clause))
    return resolvents

def negate(predicate):
    """Negate a predicate."""
    return ('not', predicate) if isinstance(predicate, str) else predicate[1]

def apply_substitution(clause, substitutions):
    """Apply substitutions to a clause."""
    return {apply_substitution_to_predicate(pred, substitutions) for pred in clause}

def apply_substitution_to_predicate(predicate, substitutions):
    """Apply substitution to a single predicate."""
    if isinstance(predicate, str):
        return substitutions.get(predicate, predicate)
    else:
        return (predicate[0],) + tuple(substitutions.get(arg, arg) for arg in predicate[1:])

def resolution(knowledge_base, target_query):
    """Perform resolution to prove the query."""
    negated_query = frozenset({negate(target_query)})
    clauses = knowledge_base | {negated_query}
    new_clauses = set()

    while True:
        for clause1, clause2 in combinations(clauses, 2):
            resolvents = resolve(clause1, clause2)
            if frozenset() in resolvents:
                return True
            new_clauses.update(resolvents)
        if new_clauses.issubset(clauses):
            return False
        clauses |= new_clauses

knowledge_base = {
    frozenset({('Mother', 'Leela', 'Oshin')}),
    frozenset({('Alive', 'Leela')}),
    frozenset({('not', 'Mother', 'X', 'Y')}),
    frozenset({('Parent', 'X', 'Y')}),
    frozenset({('not', 'Parent', 'W', 'Z')}),
    frozenset({('not', 'Alive', 'W', 'Z')}),
    frozenset({('Older', 'W', 'Z')}),
}

# Query: Prove Leela is older than Oshin
target_query = ('Older', 'Leela', 'Oshin')

# Run resolution
proof_result = resolution(knowledge_base, target_query)
if proof_result:
    print("Proved by resolution: Hence, Leela is older than Oshin.")
else:
    print("Cannot prove. Leela is not older than Oshin.")


Proved by resolution: Hence, Leela is older than Oshin.
